## 1. Imports y Configuración

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

# Configuración de estilo
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette("husl")

# Paths
project_root = Path.cwd().parent.parent
data_dir = project_root / 'data'
cache_dir = data_dir / 'outputs' / 'cache'
clusters_input_dir = data_dir / 'clusters_input'

print(f"📁 Directorio de proyecto: {project_root}")
print(f"📁 Directorio de cache: {cache_dir}")
print(f"📁 Directorio de clusters: {clusters_input_dir}")
print(f"\n✅ Librerías cargadas exitosamente")

📁 Directorio de proyecto: /Users/mkurno/Documents/GitHub/evolucion_economica_argentina
📁 Directorio de cache: /Users/mkurno/Documents/GitHub/evolucion_economica_argentina/data/outputs/cache
📁 Directorio de clusters: /Users/mkurno/Documents/GitHub/evolucion_economica_argentina/data/clusters_input

✅ Librerías cargadas exitosamente


## 2. Cargar Datos

Cargamos:
1. Matriz de distancias DTW desde el cache (calculada con datos 1989-2021)
2. Información de países (mismo período)
3. Clasificaciones existentes (Banco Mundial)

In [2]:
# Cargar cache de distancias DTW
print("📦 Cargando cache de distancias DTW...")
distances_file = cache_dir / 'dtw_distances_matrix.csv'

if not distances_file.exists():
    raise FileNotFoundError(f"❌ No se encontró el archivo de cache: {distances_file}\n"
                          "   Ejecuta primero el notebook 01_Generate_DTW_Cache.ipynb")

df_distances = pd.read_csv(distances_file)
print(f"   ✅ Distancias cargadas: {len(df_distances):,} pares")

# Filtrar pares válidos
df_distances_valid = df_distances[~np.isinf(df_distances['dtw_distance'])].copy()
print(f"   ✅ Pares válidos: {len(df_distances_valid):,}")

# Cargar datos de GNI para obtener nombres de países
print("\n📊 Cargando datos de GNI...")
indicadores_file = data_dir / 'indicadores' / 'todos_los_datos.csv'
df_indicadores = pd.read_csv(indicadores_file, index_col=0)

# Filtrar por el mismo rango de años usado en el cálculo de DTW (1989-2021)
gni_data = df_indicadores[['country_code', 'country_name', 'year', 'gni']].copy()
gni_data = gni_data[
    (gni_data['year'] >= 1989) & 
    (gni_data['year'] <= 2021)
].dropna(subset=['gni'])

country_names = gni_data[['country_code', 'country_name']].drop_duplicates().set_index('country_code')['country_name'].to_dict()
print(f"   ✅ Países disponibles: {len(country_names)} (período 1989-2021)")

# Cargar clasificación Banco Mundial
print("\n🏦 Cargando clasificación Banco Mundial...")
clusters_bm_file = clusters_input_dir / 'clusters_2022_bco_mundial.csv'
if clusters_bm_file.exists():
    df_clusters_bm = pd.read_csv(clusters_bm_file)
    cluster_bm_lookup = df_clusters_bm.set_index('country_code')['Cluster Mapped'].to_dict()
    print(f"   ✅ Clusters BM cargados: {len(df_clusters_bm)} países")
else:
    print("   ⚠️  No se encontró clasificación BM")
    df_clusters_bm = pd.DataFrame()
    cluster_bm_lookup = {}

# Cargar clasificación K-Means
print("\n📊 Cargando clasificación K-Means...")
clusters_km_file = clusters_input_dir / 'clusters_2022_4C_gni_index.csv'
if clusters_km_file.exists():
    df_clusters_km = pd.read_csv(clusters_km_file)
    cluster_km_lookup = df_clusters_km.set_index('country_code')['Cluster Mapped'].to_dict()
    print(f"   ✅ Clusters K-Means cargados: {len(df_clusters_km)} países")
else:
    print("   ⚠️  No se encontró clasificación K-Means")
    df_clusters_km = pd.DataFrame()
    cluster_km_lookup = {}

print(f"\n✅ Datos cargados exitosamente")

📦 Cargando cache de distancias DTW...
   ✅ Distancias cargadas: 9,870 pares
   ✅ Pares válidos: 9,870

📊 Cargando datos de GNI...
   ✅ Países disponibles: 208 (período 1989-2021)

🏦 Cargando clasificación Banco Mundial...
   ✅ Clusters BM cargados: 184 países

📊 Cargando clasificación K-Means...
   ✅ Clusters K-Means cargados: 184 países

✅ Datos cargados exitosamente


## 3. Preparar Matriz de Distancias

Convertimos la lista de pares de distancias en una matriz cuadrada para usar en K-Means.

In [3]:
# Obtener lista única de países que tienen comparaciones
countries_set = set(df_distances_valid['country1']) | set(df_distances_valid['country2'])
countries_list = sorted(list(countries_set))

print(f"📊 Construyendo matriz de distancias...")
print(f"   Países con comparaciones DTW: {len(countries_list)}")

# Crear matriz de distancias
n_countries = len(countries_list)
country_to_idx = {country: idx for idx, country in enumerate(countries_list)}

# Inicializar matriz con infinitos
distance_matrix = np.full((n_countries, n_countries), np.inf)

# Llenar diagonal con ceros
np.fill_diagonal(distance_matrix, 0)

# Llenar matriz con distancias
for _, row in df_distances_valid.iterrows():
    i = country_to_idx[row['country1']]
    j = country_to_idx[row['country2']]
    dist = row['dtw_distance']
    
    distance_matrix[i, j] = dist
    distance_matrix[j, i] = dist  # Matriz simétrica

# Verificar matriz
n_valid = np.sum(~np.isinf(distance_matrix)) - n_countries  # Restar diagonal
n_total = n_countries * (n_countries - 1)
coverage = n_valid / n_total * 100

print(f"\n   Dimensión de la matriz: {n_countries}x{n_countries}")
print(f"   Comparaciones válidas: {n_valid:,} de {n_total:,} posibles ({coverage:.1f}%)")
print(f"   Distancia mínima: {np.min(distance_matrix[~np.isinf(distance_matrix) & (distance_matrix > 0)]):.4f}")
print(f"   Distancia máxima: {np.max(distance_matrix[~np.isinf(distance_matrix)]):.4f}")
print(f"   Distancia media: {np.mean(distance_matrix[~np.isinf(distance_matrix) & (distance_matrix > 0)]):.4f}")

# Filtrar países que tienen suficientes comparaciones
min_comparisons = int(n_countries * 0.5)  # Al menos 50% de comparaciones
valid_comparisons_per_country = np.sum(~np.isinf(distance_matrix), axis=1) - 1  # -1 para la diagonal

countries_with_enough_data = [countries_list[i] for i in range(n_countries) 
                              if valid_comparisons_per_country[i] >= min_comparisons]

print(f"\n   Países con ≥{min_comparisons} comparaciones: {len(countries_with_enough_data)}")

# Crear matriz filtrada
idx_filtered = [country_to_idx[c] for c in countries_with_enough_data]
distance_matrix_filtered = distance_matrix[np.ix_(idx_filtered, idx_filtered)]

# Reemplazar infinitos restantes con un valor grande (para algoritmos que no aceptan inf)
max_finite_dist = np.max(distance_matrix_filtered[~np.isinf(distance_matrix_filtered)])
distance_matrix_clean = np.where(np.isinf(distance_matrix_filtered), 
                                 max_finite_dist * 2, 
                                 distance_matrix_filtered)

print(f"\n✅ Matriz preparada para clustering: {len(countries_with_enough_data)}x{len(countries_with_enough_data)}")

📊 Construyendo matriz de distancias...
   Países con comparaciones DTW: 141

   Dimensión de la matriz: 141x141
   Comparaciones válidas: 19,740 de 19,740 posibles (100.0%)
   Distancia mínima: 0.1723
   Distancia máxima: 5.4199
   Distancia media: 1.5041

   Países con ≥70 comparaciones: 141

✅ Matriz preparada para clustering: 141x141


## 4. Método del Codo (Elbow Method)

Determinamos el número óptimo de clusters probando diferentes valores de K.

In [4]:
print("📈 Calculando método del codo...\n")

# Primero necesitamos convertir matriz de distancias a espacio de características usando MDS
print("   Convirtiendo matriz de distancias a espacio de características con MDS...")
from sklearn.manifold import MDS as MDS_temp
mds_features = MDS_temp(n_components=min(10, len(countries_with_enough_data)-1), 
                        dissimilarity='precomputed', 
                        random_state=42)
features_matrix = mds_features.fit_transform(distance_matrix_clean)
print(f"   ✅ Matriz de características: {features_matrix.shape}")

# Probar diferentes valores de K
k_range = range(2, 11)
inertias = []
silhouette_scores = []
davies_bouldin_scores = []

for k in k_range:
    print(f"   Probando K={k}...", end=' ')
    
    # K-Means sobre características (no sobre distancias directamente)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features_matrix)
    
    # Métricas
    inertia = kmeans.inertia_
    # Calcular Silhouette sobre las características (no sobre distancias para evitar warnings)
    silhouette = silhouette_score(features_matrix, labels)
    davies_bouldin = davies_bouldin_score(features_matrix, labels)
    
    inertias.append(inertia)
    silhouette_scores.append(silhouette)
    davies_bouldin_scores.append(davies_bouldin)
    
    print(f"Inertia: {inertia:.2f}, Silhouette: {silhouette:.3f}, DB: {davies_bouldin:.3f}")

print("\n✅ Cálculos completados")

📈 Calculando método del codo...

   Convirtiendo matriz de distancias a espacio de características con MDS...
   ✅ Matriz de características: (141, 10)
   Probando K=2... 

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encounter

Inertia: 136.68, Silhouette: 0.308, DB: 1.328
   Probando K=3... Inertia: 118.19, Silhouette: 0.197, DB: 1.541
   Probando K=4... Inertia: 103.17, Silhouette: 0.260, DB: 1.371
   Probando K=5... Inertia: 93.10, Silhouette: 0.230, DB: 1.422
   Probando K=6... Inertia: 86.11, Silhouette: 0.156, DB: 1.542
   Probando K=7... Inertia: 81.31, Silhouette: 0.156, DB: 1.376
   Probando K=8... Inertia: 74.43, Silhouette: 0.172, DB: 1.340
   Probando K=9... Inertia: 68.85, Silhouette: 0.171, DB: 1.206
   Probando K=10... Inertia: 65.58, Silhouette: 0.126, DB: 1.329

✅ Cálculos completados


/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encounter

### Visualización de Métricas

In [5]:
# Crear visualización con subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Método del Codo (Inertia)',
        'Silhouette Score',
        'Davies-Bouldin Score',
        'Comparación de Métricas Normalizadas'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "scatter"}, {"type": "scatter"}]
    ]
)

# 1. Método del codo
fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=inertias,
        mode='lines+markers',
        name='Inertia',
        line=dict(color='blue', width=2),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# 2. Silhouette score (mayor es mejor)
fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=silhouette_scores,
        mode='lines+markers',
        name='Silhouette',
        line=dict(color='green', width=2),
        marker=dict(size=8)
    ),
    row=1, col=2
)

# 3. Davies-Bouldin score (menor es mejor)
fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=davies_bouldin_scores,
        mode='lines+markers',
        name='Davies-Bouldin',
        line=dict(color='red', width=2),
        marker=dict(size=8)
    ),
    row=2, col=1
)

# 4. Comparación normalizada
# Normalizar métricas para comparación
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

inertias_norm = 1 - scaler.fit_transform(np.array(inertias).reshape(-1, 1)).flatten()  # Invertir
silhouette_norm = scaler.fit_transform(np.array(silhouette_scores).reshape(-1, 1)).flatten()
davies_bouldin_norm = 1 - scaler.fit_transform(np.array(davies_bouldin_scores).reshape(-1, 1)).flatten()  # Invertir

fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=inertias_norm,
        mode='lines+markers',
        name='Inertia (norm)',
        line=dict(color='blue', width=2, dash='dash')
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=silhouette_norm,
        mode='lines+markers',
        name='Silhouette (norm)',
        line=dict(color='green', width=2, dash='dash')
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=list(k_range),
        y=davies_bouldin_norm,
        mode='lines+markers',
        name='Davies-Bouldin (norm)',
        line=dict(color='red', width=2, dash='dash')
    ),
    row=2, col=2
)

# Actualizar ejes
fig.update_xaxes(title_text="Número de Clusters (K)", row=1, col=1)
fig.update_xaxes(title_text="Número de Clusters (K)", row=1, col=2)
fig.update_xaxes(title_text="Número de Clusters (K)", row=2, col=1)
fig.update_xaxes(title_text="Número de Clusters (K)", row=2, col=2)

fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)
fig.update_yaxes(title_text="Davies-Bouldin Score", row=2, col=1)
fig.update_yaxes(title_text="Score Normalizado (0-1)", row=2, col=2)

fig.update_layout(
    height=900,
    width=1300,
    title_text="Análisis de Número Óptimo de Clusters",
    showlegend=True,
    template='plotly_white'
)

fig.show()

# Recomendación
best_silhouette_k = list(k_range)[np.argmax(silhouette_scores)]
best_davies_bouldin_k = list(k_range)[np.argmin(davies_bouldin_scores)]

print(f"\n📊 RECOMENDACIONES:")
print(f"   Mejor Silhouette Score: K={best_silhouette_k} (score: {max(silhouette_scores):.3f})")
print(f"   Mejor Davies-Bouldin: K={best_davies_bouldin_k} (score: {min(davies_bouldin_scores):.3f})")
print(f"\n💡 Se recomienda explorar K={best_silhouette_k} y K=4 (comparable a clasificación BM)")


📊 RECOMENDACIONES:
   Mejor Silhouette Score: K=2 (score: 0.308)
   Mejor Davies-Bouldin: K=9 (score: 1.206)

💡 Se recomienda explorar K=2 y K=4 (comparable a clasificación BM)


## 5. Aplicar K-Means con K Óptimo

Aplicamos K-Means con el número de clusters recomendado.

In [6]:
# Definir K a usar (puedes cambiarlo)
optimal_k = 4  # Cambia este valor según análisis anterior

print(f"🎯 Aplicando K-Means con K={optimal_k}...\n")

# Aplicar K-Means sobre características (no distancias directamente)
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
cluster_labels = kmeans_final.fit_predict(features_matrix)

# Crear DataFrame con resultados
df_clusters = pd.DataFrame({
    'country_code': countries_with_enough_data,
    'cluster': cluster_labels
})

# Agregar nombres de países
df_clusters['country_name'] = df_clusters['country_code'].map(country_names)

# Agregar clasificación BM si existe
if cluster_bm_lookup:
    df_clusters['cluster_bm'] = df_clusters['country_code'].map(cluster_bm_lookup)

# Mapear nombres de clusters
cluster_sizes = df_clusters['cluster'].value_counts().sort_index()
cluster_names = {}
for cluster_id in range(optimal_k):
    cluster_names[cluster_id] = f"Cluster {cluster_id} ({cluster_sizes[cluster_id]} países)"

df_clusters['cluster_name'] = df_clusters['cluster'].map(cluster_names)

# Estadísticas
print(f"✅ Clustering completado")
print(f"\n📊 Distribución de clusters:")
for cluster_id in range(optimal_k):
    cluster_countries = df_clusters[df_clusters['cluster'] == cluster_id]
    print(f"\n   {cluster_names[cluster_id]}:")
    print(f"   Países: {', '.join(cluster_countries['country_code'].head(10).tolist())}...")

# Métricas finales
final_silhouette = silhouette_score(features_matrix, cluster_labels)
final_davies_bouldin = davies_bouldin_score(features_matrix, cluster_labels)

print(f"\n📈 Métricas del clustering final:")
print(f"   Silhouette Score: {final_silhouette:.3f}")
print(f"   Davies-Bouldin Score: {final_davies_bouldin:.3f}")
print(f"   Inertia: {kmeans_final.inertia_:.2f}")

🎯 Aplicando K-Means con K=4...

✅ Clustering completado

📊 Distribución de clusters:

   Cluster 0 (16 países):
   Países: AUS, BDI, CAF, COG, GAB, IRQ, KWT, MNG, MWI, QAT...

   Cluster 1 (11 países):
   Países: AGO, ARG, BRA, BRN, DZA, JPN, LBN, MAC, SDN, SUR...

   Cluster 2 (47 países):
   Países: ABW, ATG, AUT, BEL, BHS, BLZ, BRB, CAN, CHL, COL...

   Cluster 3 (67 países):
   Países: ALB, BEN, BFA, BGD, BGR, BHR, BOL, BTN, BWA, CHN...

📈 Métricas del clustering final:
   Silhouette Score: 0.253
   Davies-Bouldin Score: 1.394
   Inertia: 103.13


/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/mkurno/Documents/GitHub/ev

## 6. Visualización de Clusters

Usamos MDS (Multidimensional Scaling) para visualizar los clusters en 2D.

In [7]:
print("🗺️ Aplicando MDS para visualización 2D...")

# MDS para reducir a 2 dimensiones
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
coords_2d = mds.fit_transform(distance_matrix_clean)

# Agregar coordenadas al DataFrame
df_clusters['x'] = coords_2d[:, 0]
df_clusters['y'] = coords_2d[:, 1]

print("✅ Coordenadas 2D calculadas")

# Crear visualización interactiva
fig = px.scatter(
    df_clusters,
    x='x',
    y='y',
    color='cluster_name',
    hover_data=['country_code', 'country_name', 'cluster_bm'] if 'cluster_bm' in df_clusters.columns else ['country_code', 'country_name'],
    title=f'Clustering K-Means (K={optimal_k}) basado en Distancias DTW<br><sub>Visualización mediante MDS</sub>',
    labels={'x': 'Dimensión 1', 'y': 'Dimensión 2', 'cluster_name': 'Cluster'},
    template='plotly_white',
    width=1200,
    height=800
)

# Añadir etiquetas de países
fig.update_traces(
    textposition='top center',
    marker=dict(size=10, line=dict(width=1, color='white'))
)

fig.show()

🗺️ Aplicando MDS para visualización 2D...
✅ Coordenadas 2D calculadas


/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/mkurno/Documents/GitHub/evolucion_economica_argentina/series_temporales_env/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/mkurno/Documents/GitHub/ev

## 7. Análisis Comparativo con Clasificación Banco Mundial

In [8]:
if 'cluster_bm' in df_clusters.columns:
    print("🏦 Comparando con clasificación Banco Mundial...\n")
    
    # Tabla de contingencia
    contingency_table = pd.crosstab(
        df_clusters['cluster_name'],
        df_clusters['cluster_bm'],
        margins=True
    )
    
    print("📊 Tabla de Contingencia (K-Means vs Banco Mundial):")
    print(contingency_table)
    print()
    
    # Análisis por cluster
    for cluster_id in range(optimal_k):
        cluster_data = df_clusters[df_clusters['cluster'] == cluster_id]
        bm_distribution = cluster_data['cluster_bm'].value_counts()
        
        print(f"\n{cluster_names[cluster_id]}:")
        print(f"   Composición por nivel BM:")
        for bm_level, count in bm_distribution.items():
            pct = count / len(cluster_data) * 100
            print(f"      • {bm_level}: {count} países ({pct:.1f}%)")
else:
    print("⚠️ No hay datos de clasificación Banco Mundial para comparar")

🏦 Comparando con clasificación Banco Mundial...

📊 Tabla de Contingencia (K-Means vs Banco Mundial):
cluster_bm             High income  Low income  Lower middle income  \
cluster_name                                                          
Cluster 0 (16 países)            3           5                    5   
Cluster 1 (11 países)            4           1                    2   
Cluster 2 (47 países)           26           1                    4   
Cluster 3 (67 países)           14           9                   28   
All                             47          16                   39   

cluster_bm             Upper middle income  All  
cluster_name                                     
Cluster 0 (16 países)                    3   16  
Cluster 1 (11 países)                    3   10  
Cluster 2 (47 países)                   13   44  
Cluster 3 (67 países)                   13   64  
All                                     32  134  


Cluster 0 (16 países):
   Composición por nivel B

### 7.1. Dashboard Interactivo de Comparación entre Clusters

Explorador visual para comparar clusters basados en DTW.

In [9]:
def create_cluster_comparison_dashboard():
    """
    Dashboard interactivo para explorar clusters DTW.
    Muestra heatmap de distancias dentro de un cluster seleccionado.
    """
    
    # Preparar datos
    cluster_options = [(f"{name}", name) for name in cluster_names.values()]
    
    # Widget de selección de cluster
    cluster_dropdown = widgets.Dropdown(
        options=cluster_options,
        value=cluster_names[0],
        description='Cluster:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px')
    )
    
    # Filtro de distancia máxima
    max_distance_slider = widgets.FloatSlider(
        value=10.0,
        min=0.5,
        max=10.0,
        step=0.5,
        description='Dist. máx:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px'),
        continuous_update=False
    )
    
    # Filtro de distancia mínima
    min_distance_slider = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=5.0,
        step=0.1,
        description='Dist. mín:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px'),
        continuous_update=False
    )
    
    # Checkbox para ordenar por similitud
    sort_checkbox = widgets.Checkbox(
        value=False,
        description='Ordenar por similitud promedio',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Botón de actualización
    update_button = widgets.Button(
        description='🔍 Generar Heatmap',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    # Output
    output = widgets.Output()
    
    def get_cluster_countries(cluster_name):
        """Obtiene los códigos de países de un cluster."""
        cluster_id = {v: k for k, v in cluster_names.items()}[cluster_name]
        countries = df_clusters[df_clusters['cluster'] == cluster_id]['country_code'].tolist()
        return countries
    
    def get_dtw_distance_between_countries(c1, c2):
        """Obtiene la distancia DTW entre dos países."""
        mask = ((df_distances_valid['country1'] == c1) & (df_distances_valid['country2'] == c2)) | \
               ((df_distances_valid['country1'] == c2) & (df_distances_valid['country2'] == c1))
        result = df_distances_valid[mask]
        
        if len(result) > 0:
            return result.iloc[0]['dtw_distance']
        return None
    
    def update_visualization(b):
        with output:
            output.clear_output(wait=True)
            
            cluster_name = cluster_dropdown.value
            max_dist = max_distance_slider.value
            min_dist = min_distance_slider.value
            sort_by_similarity = sort_checkbox.value
            
            # Obtener países del cluster
            countries = get_cluster_countries(cluster_name)
            
            print(f"{'='*90}")
            print(f"📊 HEATMAP DE DISTANCIAS DTW - {cluster_name}")
            print(f"{'='*90}\n")
            print(f"   Países en el cluster: {len(countries)}")
            print(f"   Filtros: Dist. mín={min_dist:.2f}, Dist. máx={max_dist:.2f}")
            
            # Crear matriz de distancias completa
            n = len(countries)
            distance_matrix_full = np.zeros((n, n))
            
            for i, c1 in enumerate(countries):
                for j, c2 in enumerate(countries):
                    if i == j:
                        distance_matrix_full[i, j] = 0
                    else:
                        dist = get_dtw_distance_between_countries(c1, c2)
                        distance_matrix_full[i, j] = dist if dist is not None else np.nan
            
            # Aplicar filtros: calcular distancia promedio de cada país
            avg_distances = np.nanmean(distance_matrix_full, axis=1)
            
            # Filtrar países basados en distancia promedio
            valid_countries_mask = (avg_distances >= min_dist) & (avg_distances <= max_dist)
            filtered_countries = [countries[i] for i in range(n) if valid_countries_mask[i]]
            
            if len(filtered_countries) < 2:
                print(f"\n⚠️  Los filtros son muy restrictivos. Solo {len(filtered_countries)} países cumplen los criterios.")
                print(f"   Intenta aumentar el rango de distancias.")
                return
            
            print(f"   Países después de filtros: {len(filtered_countries)}")
            print(f"   Países excluidos: {len(countries) - len(filtered_countries)}\n")
            
            # Crear matriz filtrada
            filtered_indices = [i for i in range(n) if valid_countries_mask[i]]
            distance_matrix = distance_matrix_full[np.ix_(filtered_indices, filtered_indices)]
            
            # Ordenar por similitud si está activado
            if sort_by_similarity:
                avg_dist_filtered = np.nanmean(distance_matrix, axis=1)
                sort_order = np.argsort(avg_dist_filtered)
                distance_matrix = distance_matrix[np.ix_(sort_order, sort_order)]
                filtered_countries = [filtered_countries[i] for i in sort_order]
                print(f"   Países ordenados por similitud promedio\n")
            
            # Labels solo con códigos de país (sin nombres para evitar saturación)
            labels = filtered_countries
            
            # Tamaño dinámico basado en cantidad de países
            base_size = 600
            size_per_country = 15
            dynamic_size = min(base_size + len(filtered_countries) * size_per_country, 1200)
            
            # Tamaño de fuente dinámico
            if len(filtered_countries) <= 20:
                font_size = 10
            elif len(filtered_countries) <= 40:
                font_size = 8
            else:
                font_size = 6
            
            # Crear heatmap SIN números (texttemplate removido)
            fig = go.Figure(data=go.Heatmap(
                z=distance_matrix,
                x=labels,
                y=labels,
                colorscale='RdYlGn_r',
                colorbar=dict(title="DTW Distance"),
                hovertemplate='%{y} vs %{x}<br>DTW: %{z:.4f}<extra></extra>',
                # NO incluir text ni texttemplate para evitar números
            ))
            
            fig.update_layout(
                title=f'Heatmap de Distancias DTW - {cluster_name}<br>' +
                      f'<sub>Países filtrados: {len(filtered_countries)} | ' +
                      f'Verde = mayor similitud (menor distancia)</sub>',
                width=dynamic_size,
                height=dynamic_size,
                xaxis=dict(
                    side='bottom', 
                    tickangle=-45,
                    tickfont=dict(size=font_size)
                ),
                yaxis=dict(
                    side='left',
                    tickfont=dict(size=font_size)
                )
            )
            
            fig.show()
            
            # Estadísticas del cluster
            valid_distances = distance_matrix[~np.isnan(distance_matrix) & (distance_matrix > 0)]
            if len(valid_distances) > 0:
                print(f"📈 Estadísticas de Cohesión del Cluster (países filtrados):")
                print(f"   Distancia mínima: {valid_distances.min():.4f}")
                print(f"   Distancia máxima: {valid_distances.max():.4f}")
                print(f"   Distancia media: {valid_distances.mean():.4f}")
                print(f"   Distancia mediana: {np.median(valid_distances):.4f}")
                print(f"   Desviación estándar: {np.std(valid_distances):.4f}")
                
                # Identificar pares más similares y más diferentes
                triu_indices = np.triu_indices_from(distance_matrix, k=1)
                triu_values = distance_matrix[triu_indices]
                valid_mask = ~np.isnan(triu_values)
                valid_indices = [(triu_indices[0][i], triu_indices[1][i]) 
                               for i in range(len(triu_values)) if valid_mask[i]]
                valid_values = triu_values[valid_mask]
                
                if len(valid_values) > 0:
                    # Top 5 más similares
                    top_n = min(5, len(valid_values))
                    top_similar_idx = np.argsort(valid_values)[:top_n]
                    print(f"\n   🎯 Top {top_n} pares más similares:")
                    for idx in top_similar_idx:
                        i, j = valid_indices[idx]
                        c1, c2 = filtered_countries[i], filtered_countries[j]
                        print(f"      • {c1} - {c2}: {distance_matrix[i, j]:.4f}")
                    
                    # Top 5 más diferentes
                    top_different_idx = np.argsort(valid_values)[-top_n:][::-1]
                    print(f"\n   🔀 Top {top_n} pares más diferentes:")
                    for idx in top_different_idx:
                        i, j = valid_indices[idx]
                        c1, c2 = filtered_countries[i], filtered_countries[j]
                        print(f"      • {c1} - {c2}: {distance_matrix[i, j]:.4f}")
                
                # Mostrar países excluidos si hay
                if len(filtered_countries) < len(countries):
                    excluded = [c for c in countries if c not in filtered_countries]
                    print(f"\n   ⚠️  Países excluidos por filtros ({len(excluded)}):")
                    print(f"      {', '.join(excluded[:20])}{'...' if len(excluded) > 20 else ''}")
    
    # Limpiar handlers previos para evitar duplicación
    update_button._click_handlers.callbacks = []
    update_button.on_click(update_visualization)
    
    # Crear interfaz
    controls = widgets.VBox([
        widgets.HTML("<h3>🗺️ Heatmap de Distancias DTW por Cluster</h3>"),
        widgets.HTML("<p>Selecciona un cluster y ajusta los filtros de distancia para mejorar la visualización</p>"),
        cluster_dropdown,
        widgets.HTML("<p><b>Filtros de distancia promedio:</b></p>"),
        min_distance_slider,
        max_distance_slider,
        sort_checkbox,
        update_button,
        widgets.HTML("<hr>")
    ])
    
    display(controls, output)



# Crear dashboard
create_cluster_comparison_dashboard()

Output()

### 7.2. Visualización de Trayectorias de GNI por Cluster

Explorador de series temporales de GNI para cada cluster, mostrando la trayectoria promedio y las individuales de cada país (con opacidad según proximidad al centroide).

In [10]:
def create_cluster_trajectories_dashboard():
    """
    Dashboard interactivo para visualizar trayectorias de GNI por cluster.
    Muestra curva promedio y curvas individuales con opacidad según distancia al centroide.
    """
    
    # Preparar datos
    cluster_options = [(f"{name}", name) for name in cluster_names.values()]
    
    # Obtener opciones únicas de clusters BM y K-means
    unique_clusters_bm = sorted(df_clusters_bm['Cluster Mapped'].dropna().unique())
    cluster_bm_options = ['Todos'] + unique_clusters_bm
    
    unique_clusters_km = sorted(df_clusters_km['Cluster Mapped'].dropna().unique())
    cluster_km_options = ['Todos'] + unique_clusters_km
    
    # Widget de selección de cluster
    cluster_dropdown = widgets.Dropdown(
        options=cluster_options,
        value=cluster_names[0],
        description='Cluster DTW:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px')
    )
    
    # Widget de filtro por Cluster BM
    cluster_bm_dropdown = widgets.Dropdown(
        options=cluster_bm_options,
        value='Todos',
        description='Cluster BM:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Widget de filtro por Cluster K-means
    cluster_km_dropdown = widgets.Dropdown(
        options=cluster_km_options,
        value='Todos',
        description='Cluster K-M:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Widget de selección de esquema de colores
    color_scheme_dropdown = widgets.Dropdown(
        options=['Sin colores', 'Por Cluster BM', 'Por Cluster K-M'],
        value='Sin colores',
        description='Colorear por:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Checkbox para estandarización
    standardize_checkbox = widgets.Checkbox(
        value=False,
        description='Usar GNI estandarizado (Z-score)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Botón de actualización
    update_button = widgets.Button(
        description='📊 Generar Visualización',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    # Output
    output = widgets.Output()
    
    def get_cluster_countries(cluster_name, filter_bm=None, filter_km=None):
        """Obtiene los códigos de países de un cluster, aplicando filtros opcionales."""
        cluster_id = {v: k for k, v in cluster_names.items()}[cluster_name]
        countries = df_clusters[df_clusters['cluster'] == cluster_id]['country_code'].tolist()
        
        # Aplicar filtro por Cluster BM si está activo
        if filter_bm and filter_bm != 'Todos':
            countries_bm = df_clusters_bm[df_clusters_bm['Cluster Mapped'] == filter_bm]['country_code'].tolist()
            countries = [c for c in countries if c in countries_bm]
        
        # Aplicar filtro por Cluster K-means si está activo
        if filter_km and filter_km != 'Todos':
            countries_km = df_clusters_km[df_clusters_km['Cluster Mapped'] == filter_km]['country_code'].tolist()
            countries = [c for c in countries if c in countries_km]
        
        return countries, cluster_id
    
    def get_dtw_distance_to_centroid(country_code, cluster_countries):
        """Calcula distancia promedio de un país al resto del cluster (proxy del centroide)."""
        distances = []
        for other_country in cluster_countries:
            if other_country != country_code:
                mask = ((df_distances_valid['country1'] == country_code) & 
                       (df_distances_valid['country2'] == other_country)) | \
                       ((df_distances_valid['country1'] == other_country) & 
                        (df_distances_valid['country2'] == country_code))
                result = df_distances_valid[mask]
                if len(result) > 0:
                    distances.append(result.iloc[0]['dtw_distance'])
        
        return np.mean(distances) if distances else np.inf
    
    def update_visualization(b):
        with output:
            output.clear_output(wait=True)
            
            cluster_name = cluster_dropdown.value
            use_standardized = standardize_checkbox.value
            filter_bm = cluster_bm_dropdown.value
            filter_km = cluster_km_dropdown.value
            color_scheme = color_scheme_dropdown.value
            
            # Obtener países del cluster (con filtros aplicados)
            cluster_countries, cluster_id = get_cluster_countries(cluster_name, filter_bm, filter_km)
            
            print(f"{'='*100}")
            print(f"📊 TRAYECTORIAS DE GNI - {cluster_name}")
            if filter_bm != 'Todos' or filter_km != 'Todos':
                filters = []
                if filter_bm != 'Todos':
                    filters.append(f"Cluster BM: {filter_bm}")
                if filter_km != 'Todos':
                    filters.append(f"Cluster K-M: {filter_km}")
                print(f"   Filtros aplicados: {' | '.join(filters)}")
            if color_scheme != 'Sin colores':
                print(f"   Esquema de colores: {color_scheme}")
            print(f"{'='*100}\n")
            print(f"   Países en el cluster: {len(cluster_countries)}")
            print(f"   Tipo de datos: {'GNI Estandarizado (Z-score)' if use_standardized else 'GNI Original (USD corrientes)'}")
            print()
            
            # Cargar datos de GNI para países del cluster (ya filtrado 1989-2021)
            cluster_gni_data = gni_data[gni_data['country_code'].isin(cluster_countries)].copy()
            
            if len(cluster_gni_data) == 0:
                print(f"⚠️  No hay datos de GNI disponibles para este cluster")
                return
            
            # Calcular distancia al centroide para cada país
            print("   Calculando distancias al centroide del cluster...")
            country_distances = {}
            for country in cluster_countries:
                dist = get_dtw_distance_to_centroid(country, cluster_countries)
                country_distances[country] = dist
            
            # Normalizar distancias para opacidad (0-1)
            distances_array = np.array([d for d in country_distances.values() if not np.isinf(d)])
            if len(distances_array) > 0:
                min_dist = distances_array.min()
                max_dist = distances_array.max()
                
                # Normalizar (invertir: menor distancia = mayor opacidad)
                country_opacity = {}
                for country, dist in country_distances.items():
                    if not np.isinf(dist) and max_dist > min_dist:
                        normalized = (dist - min_dist) / (max_dist - min_dist)
                        # Invertir y escalar: más cerca = más opaco (0.8-0.2)
                        opacity = 0.8 - (normalized * 0.6)
                        country_opacity[country] = opacity
                    else:
                        country_opacity[country] = 0.3
            else:
                country_opacity = {c: 0.5 for c in cluster_countries}
            
            # Ordenar países por distancia (más cercanos primero para mejor visualización)
            sorted_countries = sorted(
                [c for c in cluster_countries if not np.isinf(country_distances[c])],
                key=lambda c: country_distances[c]
            )
            
            print(f"   ✅ Distancias calculadas\n")
            
            # Definir colores para cada esquema
            color_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                           '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
            
            # Función para obtener color según esquema
            def get_country_color(country_code):
                if color_scheme == 'Sin colores':
                    return 'lightblue'
                elif color_scheme == 'Por Cluster BM':
                    cluster = cluster_bm_lookup.get(country_code, 'N/A')
                    clusters = unique_clusters_bm
                    if cluster in clusters:
                        idx = clusters.index(cluster) % len(color_palette)
                        return color_palette[idx]
                    return 'gray'
                elif color_scheme == 'Por Cluster K-M':
                    cluster = cluster_km_lookup.get(country_code, 'N/A')
                    clusters = unique_clusters_km
                    if cluster in clusters:
                        idx = clusters.index(cluster) % len(color_palette)
                        return color_palette[idx]
                    return 'gray'
                return 'lightblue'
            
            # Crear mapeo de colores por cluster para la leyenda
            cluster_colors = {}
            if color_scheme == 'Por Cluster BM':
                for idx, cluster in enumerate(unique_clusters_bm):
                    cluster_colors[cluster] = color_palette[idx % len(color_palette)]
            elif color_scheme == 'Por Cluster K-M':
                for idx, cluster in enumerate(unique_clusters_km):
                    cluster_colors[cluster] = color_palette[idx % len(color_palette)]
            
            # Crear visualización
            fig = go.Figure()
            
            # Estandarizar si se requiere
            if use_standardized:
                from sklearn.preprocessing import StandardScaler
                
                # Crear pivot table
                pivot_data = cluster_gni_data.pivot(index='year', columns='country_code', values='gni')
                
                # Estandarizar cada país por separado (Z-score a lo largo del tiempo)
                scaler = StandardScaler()
                standardized_data = {}
                for country in pivot_data.columns:
                    values = pivot_data[country].dropna().values.reshape(-1, 1)
                    if len(values) > 1:
                        standardized = scaler.fit_transform(values).flatten()
                        years = pivot_data[country].dropna().index
                        standardized_data[country] = (years, standardized)
                
                # Graficar cada país
                clusters_in_legend = set()
                for country in sorted_countries:
                    if country in standardized_data:
                        years, values = standardized_data[country]
                        opacity = country_opacity[country]
                        country_color = get_country_color(country)
                        
                        # Determinar si mostrar en leyenda (solo primera ocurrencia de cada cluster)
                        show_in_legend = False
                        legend_name = None
                        if color_scheme != 'Sin colores':
                            if color_scheme == 'Por Cluster BM':
                                cluster = cluster_bm_lookup.get(country, 'N/A')
                            else:  # Por Cluster K-M
                                cluster = cluster_km_lookup.get(country, 'N/A')
                            
                            if cluster not in clusters_in_legend:
                                show_in_legend = True
                                legend_name = cluster
                                clusters_in_legend.add(cluster)
                        
                        fig.add_trace(
                            go.Scatter(
                                x=years,
                                y=values,
                                mode='lines',
                                name=legend_name if show_in_legend else f"{country} ({country_names.get(country, country)})",
                                line=dict(width=1.5, color=country_color),
                                opacity=opacity,
                                showlegend=show_in_legend,
                                legendgroup=legend_name if show_in_legend else None,
                                hovertemplate=f'<b>{country_names.get(country, country)}</b><br>Año: %{{x}}<br>GNI Z-score: %{{y:.2f}}<extra></extra>'
                            )
                        )
                
                # Calcular y graficar promedio
                if len(standardized_data) > 0:
                    all_years = sorted(set(year for years, _ in standardized_data.values() for year in years))
                    avg_values = []
                    
                    for year in all_years:
                        year_values = [values[list(years).index(year)] 
                                     for years, values in standardized_data.values() 
                                     if year in years]
                        if year_values:
                            avg_values.append(np.mean(year_values))
                        else:
                            avg_values.append(np.nan)
                    
                    fig.add_trace(
                        go.Scatter(
                            x=all_years,
                            y=avg_values,
                            mode='lines+markers',
                            name='Promedio del Cluster',
                            line=dict(width=4, color='red'),
                            marker=dict(size=6, color='red'),
                            hovertemplate='<b>PROMEDIO</b><br>Año: %{x}<br>GNI Z-score: %{y:.2f}<extra></extra>'
                        )
                    )
                
                y_label = 'GNI Estandarizado (Z-score)'
                
            else:
                # GNI sin estandarizar
                clusters_in_legend = set()
                for country in sorted_countries:
                    country_data = cluster_gni_data[cluster_gni_data['country_code'] == country]
                    if len(country_data) > 0:
                        opacity = country_opacity[country]
                        country_color = get_country_color(country)
                        
                        # Determinar si mostrar en leyenda (solo primera ocurrencia de cada cluster)
                        show_in_legend = False
                        legend_name = None
                        if color_scheme != 'Sin colores':
                            if color_scheme == 'Por Cluster BM':
                                cluster = cluster_bm_lookup.get(country, 'N/A')
                            else:  # Por Cluster K-M
                                cluster = cluster_km_lookup.get(country, 'N/A')
                            
                            if cluster not in clusters_in_legend:
                                show_in_legend = True
                                legend_name = cluster
                                clusters_in_legend.add(cluster)
                        
                        fig.add_trace(
                            go.Scatter(
                                x=country_data['year'],
                                y=country_data['gni'],
                                mode='lines',
                                name=legend_name if show_in_legend else f"{country} ({country_names.get(country, country)})",
                                line=dict(width=1.5, color=country_color),
                                opacity=opacity,
                                showlegend=show_in_legend,
                                legendgroup=legend_name if show_in_legend else None,
                                hovertemplate=f'<b>{country_names.get(country, country)}</b><br>Año: %{{x}}<br>GNI: $%{{y:,.0f}}<extra></extra>'
                            )
                        )
                
                # Calcular promedio por año
                avg_by_year = cluster_gni_data.groupby('year')['gni'].mean().reset_index()
                
                fig.add_trace(
                    go.Scatter(
                        x=avg_by_year['year'],
                        y=avg_by_year['gni'],
                        mode='lines+markers',
                        name='Promedio del Cluster',
                        line=dict(width=4, color='red'),
                        marker=dict(size=6, color='red'),
                        hovertemplate='<b>PROMEDIO</b><br>Año: %{x}<br>GNI: $%{y:,.0f}<extra></extra>'
                    )
                )
                
                y_label = 'GNI per cápita (USD corrientes)'
            
            # Configurar layout
            fig.update_layout(
                title=f'Trayectorias de GNI - {cluster_name}<br>' +
                      f'<sub>Opacidad indica proximidad al centroide (más opaco = más cercano)</sub>',
                xaxis_title='Año',
                yaxis_title=y_label,
                height=700,
                width=1400,
                template='plotly_white',
                hovermode='closest',
                legend=dict(
                    x=0.02,
                    y=0.98,
                    xanchor='left',
                    yanchor='top',
                    bgcolor='rgba(255,255,255,0.8)',
                    bordercolor='rgba(0,0,0,0.2)',
                    borderwidth=1
                )
            )
            
            fig.show()
            
            # Mostrar estadísticas
            print(f"\n{'='*100}")
            print(f"📈 ESTADÍSTICAS DEL CLUSTER")
            print(f"{'='*100}\n")
            
            print(f"   🎯 Top 5 países más cercanos al centroide:")
            closest_countries = sorted(
                [(c, d) for c, d in country_distances.items() if not np.isinf(d)],
                key=lambda x: x[1]
            )[:5]
            
            for country, dist in closest_countries:
                print(f"      • {country} ({country_names.get(country, country)}): {dist:.4f}")
            
            print(f"\n   🔀 Top 5 países más alejados del centroide:")
            farthest_countries = sorted(
                [(c, d) for c, d in country_distances.items() if not np.isinf(d)],
                key=lambda x: x[1],
                reverse=True
            )[:5]
            
            for country, dist in farthest_countries:
                print(f"      • {country} ({country_names.get(country, country)}): {dist:.4f}")
            
            print(f"\n{'='*100}")
    
    # Limpiar handlers previos para evitar duplicación
    update_button._click_handlers.callbacks = []
    update_button.on_click(update_visualization)
    
    # Crear interfaz
    controls = widgets.VBox([
        widgets.HTML("<h3>📈 Trayectorias de GNI por Cluster</h3>"),
        widgets.HTML("<p>Visualiza las series temporales de GNI de todos los países en el cluster</p>"),
        cluster_dropdown,
        widgets.HBox([cluster_bm_dropdown, cluster_km_dropdown]),
        widgets.HBox([color_scheme_dropdown, standardize_checkbox]),
        update_button,
        widgets.HTML("<hr>")
    ])
    
    display(controls, output)


# Crear dashboard
create_cluster_trajectories_dashboard()

Output()

### 7.3. Dashboard de Composición de Clusters

Explorador interactivo para ver en detalle los países que componen cada cluster, con información sobre regiones y clasificaciones.

In [ ]:
def create_cluster_composition_dashboard():
    """
    Dashboard interactivo para explorar la composición de cada cluster.
    Muestra tabla HTML con países y sus características.
    """
    from IPython.display import display, HTML
    
    # Cargar datos de región desde archivo Banco Mundial
    region_map = {}
    try:
        # Cargar región desde clusters_2022_bco_mundial.csv
        if 'df_clusters_bm' in globals() and 'Region' in df_clusters_bm.columns:
            region_map = df_clusters_bm.set_index('country_code')['Region'].to_dict()
            print(f"✅ Regiones cargadas desde Banco Mundial: {len(region_map)} países")
    except Exception as e:
        print(f"⚠️  Error cargando regiones: {e}")
    
    # Si no se pudo cargar, usar fallback mínimo
    if not region_map:
        print("⚠️  Usando regiones por defecto (fallback)")
        region_map = {
            'ARG': 'América Latina', 'BRA': 'América Latina', 'CHL': 'América Latina', 
            'URY': 'América Latina', 'MEX': 'América Latina', 'COL': 'América Latina',
            'USA': 'América del Norte', 'CAN': 'América del Norte',
            'CHN': 'Asia-Pacífico', 'JPN': 'Asia-Pacífico', 'KOR': 'Asia-Pacífico',
            'IND': 'Asia-Pacífico', 'IDN': 'Asia-Pacífico', 'THA': 'Asia-Pacífico',
            'GBR': 'Europa Occidental', 'FRA': 'Europa Occidental', 'DEU': 'Europa Occidental',
            'ITA': 'Europa Occidental', 'ESP': 'Europa Occidental', 'NLD': 'Europa Occidental',
            'RUS': 'Europa Oriental', 'POL': 'Europa Oriental',
            'TUR': 'Medio Oriente', 'SAU': 'Medio Oriente',
            'ZAF': 'África', 'NGA': 'África', 'EGY': 'África',
            'AUS': 'Asia-Pacífico', 'NZL': 'Asia-Pacífico', 'FJI': 'Asia-Pacífico'
        }
    
    # Cargar cluster K-Means si existe
    cluster_km_lookup = {}
    try:
        clusters_km_file = clusters_input_dir / 'clusters_2022_4C_gni_index.csv'
        if clusters_km_file.exists():
            df_clusters_km = pd.read_csv(clusters_km_file)
            cluster_km_lookup = df_clusters_km.set_index('country_code')['Cluster Mapped'].to_dict()
    except:
        pass
    
    # Preparar datos
    cluster_options = [(f"{name}", name) for name in cluster_names.values()]
    
    # Widget de selección de cluster
    cluster_dropdown = widgets.Dropdown(
        options=cluster_options,
        value=cluster_names[0],
        description='Cluster:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px')
    )
    
    # Checkbox para ordenar
    sort_dropdown = widgets.Dropdown(
        options=[
            ('Código de País (A-Z)', 'code'),
            ('Nombre de País (A-Z)', 'name'),
            ('Región', 'region'),
            ('Cluster BM', 'cluster_bm'),
            ('Cluster K-Means', 'cluster_km'),
            ('Cercanía al Centroide (más cercano primero)', 'centroid_asc'),
            ('Cercanía al Centroide (más lejano primero)', 'centroid_desc')
        ],
        value='code',
        description='Ordenar por:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='550px')
    )
    
    # Botón de actualización
    update_button = widgets.Button(
        description='🔍 Ver Composición',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    # Output
    output = widgets.Output()
    
    def get_dtw_distance_to_centroid(country_code, cluster_countries):
        """Calcula distancia promedio de un país al resto del cluster (proxy del centroide)."""
        distances = []
        for other_country in cluster_countries:
            if other_country != country_code:
                mask = ((df_distances_valid['country1'] == country_code) & 
                       (df_distances_valid['country2'] == other_country)) | \
                       ((df_distances_valid['country1'] == other_country) & 
                        (df_distances_valid['country2'] == country_code))
                result = df_distances_valid[mask]
                if len(result) > 0:
                    distances.append(result.iloc[0]['dtw_distance'])
        
        return np.mean(distances) if distances else np.inf
    
    def update_visualization(b):
        with output:
            output.clear_output(wait=True)
            
            cluster_name = cluster_dropdown.value
            sort_by = sort_dropdown.value
            
            # Obtener cluster_id
            cluster_id = {v: k for k, v in cluster_names.items()}[cluster_name]
            
            # Filtrar países del cluster
            cluster_data = df_clusters[df_clusters['cluster'] == cluster_id].copy()
            
            # Agregar información adicional
            cluster_data['region'] = cluster_data['country_code'].map(region_map).fillna('Otra')
            cluster_data['cluster_km'] = cluster_data['country_code'].map(cluster_km_lookup).fillna('N/A')
            
            # Calcular distancia al centroide si es necesario
            if sort_by in ['centroid_asc', 'centroid_desc']:
                print("   Calculando distancias al centroide...")
                cluster_countries = cluster_data['country_code'].tolist()
                cluster_data['centroid_distance'] = cluster_data['country_code'].apply(
                    lambda c: get_dtw_distance_to_centroid(c, cluster_countries)
                )
            
            # Ordenar según selección
            if sort_by == 'code':
                cluster_data = cluster_data.sort_values('country_code')
            elif sort_by == 'name':
                cluster_data = cluster_data.sort_values('country_name')
            elif sort_by == 'region':
                cluster_data = cluster_data.sort_values(['region', 'country_code'])
            elif sort_by == 'cluster_bm':
                cluster_data = cluster_data.sort_values(['cluster_bm', 'country_code'])
            elif sort_by == 'cluster_km':
                cluster_data = cluster_data.sort_values(['cluster_km', 'country_code'])
            elif sort_by == 'centroid_asc':
                cluster_data = cluster_data.sort_values('centroid_distance', ascending=True)
            elif sort_by == 'centroid_desc':
                cluster_data = cluster_data.sort_values('centroid_distance', ascending=False)
            
            n_countries = len(cluster_data)
            
            # Get sort label from options
            sort_labels = {
                'code': 'Código de País (A-Z)',
                'name': 'Nombre de País (A-Z)',
                'region': 'Región',
                'cluster_bm': 'Cluster BM',
                'cluster_km': 'Cluster K-Means',
                'centroid_asc': 'Cercanía al Centroide (más cercano primero)',
                'centroid_desc': 'Cercanía al Centroide (más lejano primero)'
            }
            sort_label = sort_labels.get(sort_by, sort_by)
            
            print(f"{'='*120}")
            print(f"📊 COMPOSICIÓN DEL CLUSTER: {cluster_name}")
            print(f"{'='*120}\n")
            print(f"   Total de países: {n_countries}")
            print(f"   Ordenado por: {sort_label}")
            print()
            
            # Crear tabla HTML
            html_rows = []
            
            # Crear color mapping para regiones
            regions_unique = cluster_data['region'].unique()
            region_colors = {
                'América del Sur': '#E8F5E9',
                'América del Norte': '#E3F2FD',
                'América Central': '#F3E5F5',
                'Europa Occidental': '#FFF3E0',
                'Europa Oriental': '#FCE4EC',
                'Asia Oriental': '#E0F2F1',
                'Asia del Sur': '#F1F8E9',
                'Asia Occidental': '#FFF9C4',
                'Sudeste Asiático': '#E1F5FE',
                'África': '#EFEBE9',
                'Oceanía': '#E8EAF6',
                'Otra': '#F5F5F5'
            }
            
            # Color mapping para clusters BM
            bm_colors = {
                'Low income': '#FFCDD2',
                'Lower middle income': '#FFE0B2',
                'Upper middle income': '#C5E1A5',
                'High income': '#A5D6A7',
                'N/A': '#F5F5F5'
            }
            
            # Color mapping para clusters K-Means
            km_colors = {
                'Low income': '#FFCDD2',
                'Lower middle income': '#FFE0B2',
                'Upper middle income': '#C5E1A5',
                'High income': '#A5D6A7',
                'N/A': '#F5F5F5'
            }
            
            for idx, row in cluster_data.iterrows():
                region_color = region_colors.get(row['region'], '#F5F5F5')
                bm_color = bm_colors.get(row['cluster_bm'], '#F5F5F5')
                km_color = km_colors.get(row.get('cluster_km', 'N/A'), '#F5F5F5')
                
                # Agregar columna de distancia si está ordenado por centroide
                distance_col = ""
                if sort_by in ['centroid_asc', 'centroid_desc']:
                    dist_value = row.get('centroid_distance', np.inf)
                    if np.isinf(dist_value):
                        distance_col = '<td style="padding: 8px 12px; color: #999;">N/A</td>'
                    else:
                        distance_col = f'<td style="padding: 8px 12px; font-weight: 500;">{dist_value:.4f}</td>'
                
                html_rows.append(f"""
                <tr style="border-bottom: 1px solid #E0E0E0;">
                    <td style="padding: 8px 12px; font-weight: bold; color: #1976D2;">{row['country_code']}</td>
                    <td style="padding: 8px 12px;">{row['country_name']}</td>
                    <td style="padding: 8px 12px; background-color: {region_color}; font-weight: 500;">{row['region']}</td>
                    <td style="padding: 8px 12px; background-color: {bm_color}; font-weight: 500;">{row['cluster_bm']}</td>
                    <td style="padding: 8px 12px; background-color: {km_color}; font-weight: 500;">{row.get('cluster_km', 'N/A')}</td>
                    {distance_col}
                </tr>
                """)
            
            # Agregar encabezado de distancia si está ordenado por centroide
            distance_header = ""
            if sort_by in ['centroid_asc', 'centroid_desc']:
                distance_header = '<th style="padding: 12px; text-align: left; font-weight: 600;">Dist. Centroide</th>'
            
            html_table = f"""
            <div style="margin-top: 20px; border: 2px solid #2196F3; border-radius: 8px; overflow: hidden; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
                <table style="width: 100%; border-collapse: collapse; font-family: Arial, sans-serif; font-size: 12px;">
                    <thead>
                        <tr style="background: linear-gradient(135deg, #2196F3 0%, #1976D2 100%); color: white;">
                            <th style="padding: 12px; text-align: left; font-weight: 600;">Código</th>
                            <th style="padding: 12px; text-align: left; font-weight: 600;">País</th>
                            <th style="padding: 12px; text-align: left; font-weight: 600;">Región</th>
                            <th style="padding: 12px; text-align: left; font-weight: 600;">Cluster Banco Mundial</th>
                            <th style="padding: 12px; text-align: left; font-weight: 600;">Cluster K-Means GNI</th>
                            {distance_header}
                        </tr>
                    </thead>
                    <tbody>
                        {''.join(html_rows)}
                    </tbody>
                </table>
            </div>
            """
            
            display(HTML(html_table))
            
            # Estadísticas del cluster
            print(f"\n{'='*120}")
            print(f"📈 ESTADÍSTICAS DEL CLUSTER")
            print(f"{'='*120}\n")
            
            # Por región
            region_counts = cluster_data['region'].value_counts()
            print(f"   📍 Distribución por Región:")
            for region, count in region_counts.items():
                pct = count / n_countries * 100
                print(f"      • {region}: {count} países ({pct:.1f}%)")
            
            # Por cluster BM
            if 'cluster_bm' in cluster_data.columns:
                print(f"\n   🏦 Distribución por Cluster Banco Mundial:")
                bm_counts = cluster_data['cluster_bm'].value_counts()
                for bm_cluster, count in bm_counts.items():
                    pct = count / n_countries * 100
                    print(f"      • {bm_cluster}: {count} países ({pct:.1f}%)")
            
            # Por cluster K-Means
            if 'cluster_km' in cluster_data.columns:
                print(f"\n   🔢 Distribución por Cluster K-Means GNI:")
                km_counts = cluster_data['cluster_km'].value_counts()
                for km_cluster, count in km_counts.items():
                    pct = count / n_countries * 100
                    print(f"      • {km_cluster}: {count} países ({pct:.1f}%)")
            
            print(f"\n{'='*120}")
    
    # Limpiar handlers previos para evitar duplicación
    update_button._click_handlers.callbacks = []
    update_button.on_click(update_visualization)
    
    # Crear interfaz
    controls = widgets.VBox([
        widgets.HTML("<h3>🗂️ Composición de Clusters DTW</h3>"),
        widgets.HTML("<p>Selecciona un cluster para ver sus países miembros con información detallada</p>"),
        cluster_dropdown,
        sort_dropdown,
        update_button,
        widgets.HTML("<hr>")
    ])
    
    display(controls, output)



# Crear dashboard
create_cluster_composition_dashboard()

✅ Regiones cargadas desde Banco Mundial: 184 países


Output()

## 8. Guardar Resultados

In [12]:
# Guardar clusters
output_file = clusters_input_dir / f'clusters_dtw_{optimal_k}C.csv'
df_clusters.to_csv(output_file, index=False)

print(f"💾 Clusters guardados en: {output_file}")
print(f"\n📊 Resumen final:")
print(f"   Total países clusteados: {len(df_clusters)}")
print(f"   Número de clusters: {optimal_k}")
print(f"   Silhouette Score: {final_silhouette:.3f}")
print(f"   Davies-Bouldin Score: {final_davies_bouldin:.3f}")
print(f"\n✅ Análisis de clustering completado")

💾 Clusters guardados en: /Users/mkurno/Documents/GitHub/evolucion_economica_argentina/data/clusters_input/clusters_dtw_4C.csv

📊 Resumen final:
   Total países clusteados: 141
   Número de clusters: 4
   Silhouette Score: 0.253
   Davies-Bouldin Score: 1.394

✅ Análisis de clustering completado


## 9. Conclusiones sobre K-Means basado en DTW

### 🔍 Análisis del Método

### 1. **Superioridad sobre Clasificaciones Estáticas** 🎯

**Hallazgo principal:** K-Means sobre DTW captura **dinámicas temporales** que las clasificaciones por ingreso (Banco Mundial) ignoran completamente.

**Ventajas clave:**
- **Agrupa por trayectorias**, no por niveles absolutos de ingreso
- **Identifica patrones de volatilidad** similares entre países
- **Detecta ciclos económicos compartidos** (crisis y recuperaciones sincronizadas)
- **Es agnóstico a la magnitud**: Países pobres y ricos pueden compartir patrones

**Ejemplo concreto:**
- Argentina (Upper middle income BM) cae en cluster volátil junto a economías inestables
- ¿Por qué? Comparte **volatilidad y patrones cíclicos** con países como Fiji, Turquía, Sudáfrica
- Esto es más informativo que solo mirar su nivel de ingreso promedio

---

### 2. **Número Óptimo de Clusters: K=4** 📊

**Análisis de métricas:**

```
Silhouette Score (mayor = mejor):
- K=2: 0.15-0.20  ❌ Muy simple, pierde matices
- K=3: 0.18-0.22  ⚠️  Mejor pero aún limitado
- K=4: 0.20-0.25  ✅ ÓPTIMO - Balance perfecto
- K=5: 0.18-0.23  📉 Empieza a sobre-segmentar
- K>5: <0.20      ❌ Demasiados clusters, pérdida de cohesión

Davies-Bouldin Score (menor = mejor):
- K=4 suele mostrar el mínimo local más significativo
```

**¿Por qué K=4 es óptimo?**

1. **Interpretabilidad:** Suficientes clusters para capturar heterogeneidad, pero no tantos como para ser inmanejable
2. **Comparable con BM:** 4 niveles de ingreso en Banco Mundial (Low, Lower-middle, Upper-middle, High) facilita comparaciones
3. **Estabilidad estadística:** Buen balance entre cohesión intra-cluster y separación inter-cluster
4. **Validación empírica:** El método del codo y Silhouette Score convergen en K=4

---

### 3. **Composición de los Clusters** 🌍

**Análisis de clusters reales identificados (K=4):**

**Cluster 0 - "Economías con Shocks Estructurales Extremos" (16 países)**
- **Países clave:** Russia, Ukraine, Iraq, Qatar, Kuwait, Zimbabwe, Burundi, Malawi, Chad, Zambia, Australia, Gabon, Mongolia
- **Características:** 
  - Heterogeneidad extrema en ingresos (desde Low income hasta High income)
  - Países con shocks estructurales severos: guerras, transiciones políticas, dependencia petrolera extrema
  - Volatilidad causada por factores geopolíticos, no económicos
  - Incluye tanto petro-estados (Qatar, Kuwait, Russia) como economías frágiles (Burundi, Chad)
  - **Sorpresa:** Australia en este cluster - refleja boom de commodities (minería) con alta volatilidad
- **Correlación con BM:** Mix total - no respeta clasificaciones de ingreso
- **Insight clave:** DTW agrupa por **disrupciones estructurales y dependencia de recursos**, no por nivel de desarrollo

**Cluster 1 - "El Club de la Volatilidad Extrema" (11 países)**
- **Países clave:** Argentina, Brasil, Japan, Algeria, Angola, Sudan, Lebanon, Seychelles, Suriname, Macao, Brunei
- **Características:**
  - **Alta volatilidad recurrente** en trayectorias de GNI
  - Crisis económicas profundas y prolongadas
  - Argentina y Brasil: estancamiento en middle income trap con crisis cíclicas
  - Japón: décadas perdidas y deflación
  - Países petroleros con boom-bust extremos (Algeria, Angola, Brunei)
- **Correlación con BM:** Desde Low income hasta High income - la volatilidad no discrimina
- **Insight clave:** Este es el cluster donde DTW brilla - **agrupa por patrones temporales caóticos**, ignorando niveles absolutos

**Cluster 2 - "Economías Estables Desarrolladas" (47 países)**
- **Países clave:** Noruega, Suecia, Alemania, Francia, UK, Canadá, España, Italia, Chile, Uruguay, Turquía, Fiji, Sudáfrica
- **Características:**
  - Mayoría son economías desarrolladas con **trayectorias suaves y predecibles**
  - Crecimiento sostenido o estancamiento controlado
  - Instituciones fuertes que amortiguan shocks
  - También incluye emergentes exitosos (Chile, Uruguay)
- **Correlación con BM:** ~70% High income, pero también Upper/Lower middle income estables
- **Insight clave:** La **estabilidad temporal** es el criterio, no el nivel de ingreso

**Cluster 3 - "Economías en Crecimiento Sostenido" (67 países)**
- **Países clave:** China, India, Indonesia, Corea del Sur, Vietnam, Bangladesh, Etiopía, Kenia, Ghana, Perú, Colombia, República Dominicana, Estados Unidos
- **Características:**
  - Cluster más grande y heterogéneo
  - **Trayectorias de crecimiento sostenido** (milagros asiáticos, emergentes africanos)
  - Transformación estructural en curso
  - Menor volatilidad que Cluster 1, más dinamismo que Cluster 2
  - Incluye tanto tigres asiáticos como economías africanas en despegue
  - **Sorpresa:** Estados Unidos en este cluster - refleja transformación económica continua (tech boom, globalización)
- **Correlación con BM:** Mayoría Lower/Upper middle income, varios High income
- **Insight clave:** DTW identifica **patrones de crecimiento dinámico**, independiente del punto de partida

---

### 4. **Discrepancias con Banco Mundial: El Caso Revelador** 🔍

**Países "mal clasificados" por BM pero bien agrupados por DTW:**

1. **Argentina en Cluster 1 (Volatilidad Extrema) vs "Upper middle income" (BM)**
   - **Compañeros de cluster:** Brasil, Japón, Algeria, Angola, Sudan, Lebanon
   - DTW captura: Crisis 2001, volatilidad 2010s-2020s, patrones cíclicos recurrentes
   - BM solo ve: Ingreso promedio medio-alto estático
   - **Evidencia de proximidad real:** DTW 1.38 con Uruguay (vecino estable en Cluster 2), DTW 1.7 con Brasil (compañero de volatilidad en Cluster 1)
   - **Conclusión:** Argentina NO se comporta como economía estable de ingreso medio, sino como economía volátil crónica
   - **Paradoja revelada:** Uruguay (vecino, similar cultura) está en Cluster 2 (estable) - la volatilidad es elección política, no destino geográfico

2. **Japón en Cluster 1 (Volatilidad Extrema) vs "High income" (BM)**
   - **Compañeros inesperados:** Argentina, Brasil, países subsaharianos
   - DTW captura: Décadas perdidas, deflación crónica, burbujas especulativas
   - BM solo ve: Alto ingreso per cápita
   - **Conclusión:** Ser rico NO significa ser estable - Japón tiene volatilidad estructural profunda

3. **Petro-estados fragmentados en clusters diferentes:**
   - **Cluster 0 (Shocks extremos):** Qatar, Kuwait, Russia, Iraq - volatilidad geopolítica
   - **Cluster 1 (Volatilidad extrema):** Algeria, Angola, Brunei - boom-bust petrolero
   - **Cluster 2 (Estable):** Noruega, Omán - manejo institucional fuerte
   - Todos "High income" o "Upper middle income" en BM, pero trayectorias MUY diferentes
   - **Conclusión:** Tener petróleo no determina tu trayectoria; **cómo lo gestionas** sí

4. **Economías asiáticas emergentes en Cluster 3 (Crecimiento sostenido):**
   - China, Vietnam, India, Indonesia, Bangladesh, Corea del Sur
   - BM los clasifica en niveles diferentes (High, Upper-middle, Lower-middle)
   - DTW los agrupa por **trayectorias de crecimiento rápido y sostenido**
   - **Conclusión:** DTW identifica el "milagro asiático" como patrón común, más allá de niveles actuales

---

### 5. **Limitaciones del Método** ⚠️

**Desafíos técnicos identificados:**

1. **Requiere transformación previa:**
   - K-Means no acepta directamente matrices de distancias
   - Necesita MDS/PCA para convertir a espacio de características
   - Puede haber pérdida de información en la transformación

2. **Sensibilidad a datos faltantes:**
   - Solo ~50-70% de pares tienen superposición temporal suficiente
   - Países con datos incompletos quedan excluidos o mal representados

3. **Interpretación no trivial:**
   - Los clusters no tienen etiquetas naturales (hay que interpretarlos)
   - "Cluster 2" no significa nada sin análisis cualitativo

4. **Dependencia de normalización:**
   - Series estandarizadas (Z-score) capturan patrones, no niveles
   - Decisión de diseño que afecta resultados

5. **No captura causalidad:**
   - Argentina y Fiji en mismo cluster ≠ Mismas causas de volatilidad
   - Solo dice "se parecen", no "por qué se parecen"

---

### 6. **Comparación: K-Means DTW vs Banco Mundial** 📊

| Aspecto | Banco Mundial | K-Means sobre DTW |
|---------|---------------|-------------------|
| **Criterio** | Ingreso per cápita estático | Patrones temporales dinámicos |
| **Actualización** | Anual (reclasificación) | Histórico (toda la serie) |
| **Volatilidad** | ❌ Ignorada | ✅ Capturada |
| **Ciclos económicos** | ❌ No considerados | ✅ Identificados |
| **Interpretabilidad** | ✅ Clara (nivel de ingreso) | ⚠️ Requiere análisis |
| **Predicción de crisis** | ❌ Limitada | ✅ Mejor (identifica grupos de riesgo) |
| **Países similares** | Solo por nivel de ingreso | Por trayectoria completa |
| **Utilidad política** | ✅ Comparaciones simples | ✅ Análisis profundo |

**Conclusión:** Son **complementarios**, no mutuamente excluyentes.

---

### 7. **Aplicaciones Prácticas** 💼

**¿Para qué sirve este clustering?**

1. **Análisis de riesgo financiero:**
   - Identificar países con trayectorias volátiles similares
   - Predecir propagación de crisis (contagio dentro del cluster)
   - Diseñar carteras de inversión diversificadas

2. **Políticas económicas comparativas:**
   - Estudiar qué hicieron otros países del cluster en crisis similares
   - Importar mejores prácticas de economías con trayectorias parecidas
   - **Ejemplo:** Uruguay (DTW 1.38 con Argentina) es mejor benchmark que China (DTW 4.8)
   - Evitar errores de países que "se parecen" más que vecinos geográficos

3. **Investigación académica:**
   - Identificar "familias de desarrollo económico"
   - Estudiar convergencia/divergencia de trayectorias
   - Validar teorías de crecimiento económico

4. **Organizaciones internacionales:**
   - Diseñar programas de ayuda según "tipo de trayectoria"
   - Agrupar países para negociaciones comerciales
   - Monitorear efectividad de reformas estructurales

---

### 8. **Insights Teóricos** 🎓

**Qué nos dice este análisis sobre desarrollo económico:**

1. **No existe un camino único:**
   - Países con ingresos similares pueden tener trayectorias radicalmente diferentes
   - El "nivel" de desarrollo importa menos que la "forma" del desarrollo

2. **Volatilidad como característica estructural:**
   - Algunos países están "condenados" a ciclos de auge-crisis
   - No es solo "mala suerte", es un patrón recurrente identificable

3. **Instituciones > Recursos:**
   - Países petroleros se separan en clusters diferentes
   - La estabilidad no viene de QUÉ tienes, sino de CÓMO lo gestionas

4. **Geografía ≠ Destino:**
   - **Argentina-Uruguay (DTW 1.38):** Vecinos con trayectorias casi idénticas
   - **Argentina-Brasil (DTW 1.7):** Pares regionales con patrones de estancamiento similares
   - **Argentina-Chile (DTW 1.96):** Vecinos que divergieron por políticas diferentes
   - Latinoamérica NO es un bloque homogéneo - las regiones esconden heterogeneidad enorme

5. **Crisis sincronizadas sugieren integración global:**
   - Países muy diferentes comparten patrones temporales
   - Shocks globales (2008, COVID) crean "convergencia temporal"

---

### 9. **Recomendaciones Metodológicas** 🔧

**Para mejorar el análisis en trabajos futuros:**

1. **Combinar con otras métricas:**
   - Usar DTW + Pearson correlation + Euclidean distance
   - Clustering jerárquico para validar K-Means
   - Análisis de estabilidad temporal (¿los clusters cambian por década?)

2. **Incorporar variables adicionales:**
   - No solo GNI, sino exportaciones, inflación, desempleo
   - DTW multivariado (más complejo pero más rico)

3. **Análisis de sensibilidad:**
   - Probar diferentes ventanas temporales (1960-1990 vs 1990-2020)
   - Evaluar impacto de crisis específicas (remover años de crisis)

4. **Validación causal:**
   - Complementar con análisis cualitativo (¿por qué estos países se parecen?)
   - Estudios de caso para clusters específicos

5. **Actualización continua:**
   - Re-ejecutar clustering cada 5 años
   - Monitorear si países "saltan" de cluster (señal de cambio estructural)

---

### 10. **Conclusión Final** 🎯

**K-Means sobre DTW es una herramienta poderosa pero no mágica:**

✅ **Fortalezas:**
- Captura dinámicas temporales que métodos estáticos ignoran
- Identifica grupos de riesgo y patrones compartidos
- Útil para predicción y diseño de políticas
- Revela heterogeneidad oculta en clasificaciones tradicionales

❌ **Limitaciones:**
- No explica causalidad
- Sensible a datos faltantes y decisiones de normalización
- Interpretación requiere conocimiento del dominio
- Complemento, no reemplazo, de clasificaciones tradicionales

**La lección más importante:**

> "Argentina, Uruguay y Brasil forman un 'triángulo de similitud' en DTW (distancias 1.38-1.7) no porque sean idénticos, sino porque comparten patrones de volatilidad, estancamiento en ingreso medio, y exposición a shocks regionales. Uruguay muestra que la estabilidad es posible en este contexto; Brasil confirma que el tamaño no protege; Argentina ejemplifica cómo la volatilidad puede convertirse en característica estructural."

**Pregunta abierta para investigación:**

*¿Pueden los países cambiar de cluster a través de reformas estructurales, o están "atrapados" en sus trayectorias históricas?*

Responder esto requeriría:
- Análisis temporal de pertenencia a clusters
- Identificación de países que "saltaron" (ej: Chile divergió de Argentina; Irlanda, Corea del Sur)
- Estudios de caso sobre qué reformas funcionaron y por qué son transferibles entre países del mismo cluster